## MLBookCamp Homework 3

- [Course page](https://datatalks.club/courses/2021-winter-ml-zoomcamp.html)

- [Homework page](https://github.com/alexeygrigorev/mlbookcamp-code/blob/master/course-zoomcamp/03-classification/homework.md)

### Dataset

In this homework, we will continue the New York City Airbnb Open Data. You can take it from [Kaggle](https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data?select=AB_NYC_2019.csv) or download from [here](https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv) if you don't want to sign up to Kaggle.  
\
We'll keep working with the 'price' variable, and we'll transform it to a classification task.

In [1]:
# Import basic modules beforehand
import os
import numpy as np
import pandas as pd
import sklearn



In [2]:
os.listdir()

['.ipynb_checkpoints',
 'airbnb.csv',
 'ch-3-telecom-churn.ipynb',
 'churn.csv',
 'hw-3-classification.ipynb',
 'metrics.ipynb']

In [3]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv -O 'airbnb.csv'

--2021-09-26 21:17:24--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7077973 (6.8M) [text/plain]
Saving to: ‘airbnb.csv’

airbnb.csv          100%[===================>]   6.75M  6.42MB/s    in 1.1s    

2021-09-26 21:17:28 (6.42 MB/s) - ‘airbnb.csv’ saved [7077973/7077973]



In [4]:
os.listdir()

['.ipynb_checkpoints',
 'airbnb.csv',
 'ch-3-telecom-churn.ipynb',
 'churn.csv',
 'hw-3-classification.ipynb',
 'metrics.ipynb']

In [5]:
df = pd.read_csv('airbnb.csv')

In [6]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


### Features

For the rest of the homework, you'll need to use the features from the previous homework with additional two 'neighbourhood_group' and 'room_type'. So the whole feature set will be set as follows:

    'neighbourhood_group',
    'room_type',
    'latitude',
    'longitude',
    'price',
    'minimum_nights',
    'number_of_reviews',
    'reviews_per_month',
    'calculated_host_listings_count',
    'availability_365'

Select only them and fill in the missing values with 0.

In [7]:
# list of features to be used
features = [
'neighbourhood_group',
'room_type',
'latitude',
'longitude',
'minimum_nights',
'number_of_reviews',
'reviews_per_month',
'calculated_host_listings_count',
'availability_365',
'price'
]

len(features)

10

In [8]:
# Fetching desired df
abnb_df = df[features]
abnb_df.head()

,neighbourhood_group,room_type,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,price
0,Brooklyn,Private room,40.64749,-73.97237,1,9,0.21,6,365,149
1,Manhattan,Entire home/apt,40.75362,-73.98377,1,45,0.38,2,355,225
2,Manhattan,Private room,40.80902,-73.94190,3,0,NaN,1,365,150
3,Brooklyn,Entire home/apt,40.68514,-73.95976,1,270,4.64,1,194,89
4,Manhattan,Entire home/apt,40.79851,-73.94399,10,9,0.10,1,0,80


In [9]:
abnb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   neighbourhood_group             48895 non-null  object 
 1   room_type                       48895 non-null  object 
 2   latitude                        48895 non-null  float64
 3   longitude                       48895 non-null  float64
 4   minimum_nights                  48895 non-null  int64  
 5   number_of_reviews               48895 non-null  int64  
 6   reviews_per_month               38843 non-null  float64
 7   calculated_host_listings_count  48895 non-null  int64  
 8   availability_365                48895 non-null  int64  
 9   price                           48895 non-null  int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 3.7+ MB


> Select only them and fill in the missing values with 0.

From df info, we can observe there is one column: `reviews_per_month` with NaN values

In [10]:
# Fill nan values with 0
abnb_df['reviews_per_month'] = abnb_df['reviews_per_month'].fillna(0)

abnb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   neighbourhood_group             48895 non-null  object 
 1   room_type                       48895 non-null  object 
 2   latitude                        48895 non-null  float64
 3   longitude                       48895 non-null  float64
 4   minimum_nights                  48895 non-null  int64  
 5   number_of_reviews               48895 non-null  int64  
 6   reviews_per_month               48895 non-null  float64
 7   calculated_host_listings_count  48895 non-null  int64  
 8   availability_365                48895 non-null  int64  
 9   price                           48895 non-null  int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 3.7+ MB


/tmp/ipykernel_298/3957807480.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abnb_df['reviews_per_month'] = abnb_df['reviews_per_month'].fillna(0)


#### Question 1

What is the most frequent observation (mode) for the column 'neighbourhood_group'?
> Manhattan - 21661

In [11]:
# Fetch mode using value_counts
abnb_df['neighbourhood_group'].value_counts(ascending= False)

Manhattan        21661
Brooklyn         20104
Queens            5666
Bronx             1091
Staten Island      373
Name: neighbourhood_group, dtype: int64

### Split the data

   -  Split your data in train/val/test sets, with 60%/20%/20% distribution.  
    
   -  Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.  
    
   -  Make sure that the target value ('price') is not in your dataframe.


In [12]:
# Slice price column 
target = abnb_df['price'].to_frame()

# Drop price column from df
abnb_df.drop(columns='price', inplace= True)

/home/ada4ubuntu/anaconda3/envs/mlcamp/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


### Make price binary

   -  We need to turn the price variable from numeric into binary.
   -  Let's create a variable above_average which is 1 if the price is above (or equal to) 152.
    

In [13]:
# Binarize price
target['above_avg'] = (target['price'] >= 152).astype(int)
target.head()

,price,above_avg
0,149,0
1,225,1
2,150,0
3,89,0
4,80,0


In [14]:
from sklearn.model_selection import train_test_split
# Split data
x_train, x_test, y_train, y_test = train_test_split(abnb_df, target, test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=42)

In [15]:
print(
f'Train size: {len(x_train)}, {len(y_train)}\n'
f'Val size: {len(x_val)}, {len(y_val)}\n'
f'Test size: {len(x_test)}, {len(y_test)}'
)

Train size: 29337, 29337
Val size: 9779, 9779
Test size: 9779, 9779


### Question 2

- Create the correlation matrix for the numerical features of your train dataset.
    - In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.
- What are the two features that have the biggest correlation in this dataset?  


> `reviews_per_month` and `number_of_reviews` -> 0.549792


In [16]:
# Evaluate correlation
corr = x_train.corr()
corr.style.background_gradient(cmap='coolwarm')

,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
latitude,1.000000,0.080301,0.027441,-0.006246,-0.007159,0.019375,-0.005891
longitude,0.080301,1.000000,-0.060660,0.055084,0.134642,-0.117041,0.083666
minimum_nights,0.027441,-0.060660,1.000000,-0.076020,-0.120703,0.118647,0.138901
number_of_reviews,-0.006246,0.055084,-0.076020,1.000000,0.590374,-0.073167,0.174477
reviews_per_month,-0.007159,0.134642,-0.120703,0.590374,1.000000,-0.048767,0.165376
calculated_host_listings_count,0.019375,-0.117041,0.118647,-0.073167,-0.048767,1.000000,0.225913
availability_365,-0.005891,0.083666,0.138901,0.174477,0.165376,0.225913,1.000000


In [17]:
# Fill diagonal and upper half with NaNs
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
corr[mask] = np.nan
(corr
 .style
 .background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)
 .highlight_null(null_color='#f1f1f1')  # Color NaNs grey
 .set_precision(2))

/tmp/ipykernel_298/3700738594.py:5: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  (corr


,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
latitude,nan,nan,nan,nan,nan,nan,nan
longitude,0.08,nan,nan,nan,nan,nan,nan
minimum_nights,0.03,-0.06,nan,nan,nan,nan,nan
number_of_reviews,-0.01,0.06,-0.08,nan,nan,nan,nan
reviews_per_month,-0.01,0.13,-0.12,0.59,nan,nan,nan
calculated_host_listings_count,0.02,-0.12,0.12,-0.07,-0.05,nan,nan
availability_365,-0.01,0.08,0.14,0.17,0.17,0.23,nan


### Question 3

- Calculate the mutual information score with the (binarized) price for the two categorical variables that we have. Use the training set only.
- Which of these two variables has bigger score?
- Round it to 2 decimal digits using round(score, 2)

> Room type has bigger score (0.14)

In [18]:
from sklearn.metrics import mutual_info_score

# Fetch numerical columns
num_cols = list(x_train.select_dtypes(include=['int64', 'float64']).columns)
print(num_cols)

# Fetch categorical columns
cat_cols = list(x_train.select_dtypes(include=['object']).columns)
print(cat_cols)

['latitude', 'longitude', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']
['neighbourhood_group', 'room_type']


In [19]:
# Evaluate mutual info score
mi_ng = mutual_info_score(x_train['neighbourhood_group'], y_train['above_avg']).round(2)
mi_rt = mutual_info_score(x_train['room_type'], y_train['above_avg']).round(2)

In [20]:
print(
f'Mutual information of neighbourhood group with target: {mi_ng}\n'
f'Mutual information of room type with target: {mi_rt}\n'
)

Mutual information of neighbourhood group with target: 0.05
Mutual information of room type with target: 0.14



### Question 4

- Now let's train a logistic regression
- Remember that we have two categorical variables in the data. Include them using one-hot encoding.
- Fit the model on the training dataset.
    - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
- Calculate the accuracy on the validation dataset and rount it to 2 decimal digits.

> Validation accuracy is 0.79


In [21]:
from sklearn.preprocessing import OneHotEncoder

# Trying OHE for categorical columns
# Create instance of OHE
ohe = OneHotEncoder(sparse= False)

# Fitting categorical columns to encoder instances
ohe.fit(x_train[cat_cols])

# Fetching encoded column names
enc_cols = list(ohe.get_feature_names(cat_cols))
print(f'Encoded columns: {enc_cols}')

Encoded columns: ['neighbourhood_group_Bronx', 'neighbourhood_group_Brooklyn', 'neighbourhood_group_Manhattan', 'neighbourhood_group_Queens', 'neighbourhood_group_Staten Island', 'room_type_Entire home/apt', 'room_type_Private room', 'room_type_Shared room']


In [22]:
x_train[enc_cols] = ohe.transform(x_train[cat_cols])
x_val[enc_cols] = ohe.transform(x_val[cat_cols])
x_test[enc_cols] = ohe.transform(x_test[cat_cols])

/home/ada4ubuntu/anaconda3/envs/mlcamp/lib/python3.8/site-packages/pandas/core/frame.py:3673: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [24]:
x_train.isna().sum()

neighbourhood_group                  0
room_type                            0
latitude                             0
longitude                            0
minimum_nights                       0
number_of_reviews                    0
reviews_per_month                    0
calculated_host_listings_count       0
availability_365                     0
neighbourhood_group_Bronx            0
neighbourhood_group_Brooklyn         0
neighbourhood_group_Manhattan        0
neighbourhood_group_Queens           0
neighbourhood_group_Staten Island    0
room_type_Entire home/apt            0
room_type_Private room               0
room_type_Shared room                0
dtype: int64

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
lr.fit(x_train[num_cols+enc_cols], y_train['above_avg'])

/home/ada4ubuntu/anaconda3/envs/mlcamp/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [32]:
y_pred = lr.predict(x_val[num_cols+enc_cols])
val_acc = accuracy_score(y_val['above_avg'], y_pred)
print(f'Validation accuracy is {round(val_acc, 2)}')

Validation accuracy is 0.79


### Question 5

- We have 9 features: 7 numerical features and 2 categorical.  

- Let's find the least useful one using the feature elimination technique.
- Train a model with all these features (using the same parameters as in Q4).
- Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
- For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
- Which of following feature has the smallest difference?
        neighbourhood_group
        room_type
        number_of_reviews
        reviews_per_month

note: the difference doesn't have to be positive


In [38]:
list(zip(list(), lr.coef_))

[('latitude',
  array([-0.22829667, -0.09515034, -0.01245149, -0.00326177, -0.04173265,
          0.00396225,  0.00292928, -0.39797654,  0.08051472,  1.24402499,
         -0.79766009, -0.1315313 ,  1.63023181, -1.16015477, -0.47270526]))]

In [84]:
pd.DataFrame(data= {'feature': num_cols+enc_cols, 'coef': abs(lr.coef_[0])}).sort_values(by='coef',ascending=False)

,feature,coef
12,room_type_Entire home/apt,1.630232
9,neighbourhood_group_Manhattan,1.244025
13,room_type_Private room,1.160155
10,neighbourhood_group_Queens,0.797660
14,room_type_Shared room,0.472705
7,neighbourhood_group_Bronx,0.397977
0,latitude,0.228297
11,neighbourhood_group_Staten Island,0.131531
1,longitude,0.095150
8,neighbourhood_group_Brooklyn,0.080515


15

15